# SIT742: Modern Data Science
**(Module: Big Data)**

---
- Materials in this module include resources collected from various open-source online repositories.
- You are free to use, change and distribute this package.
- If you found any issue/bug for this document, please submit an issue at [tulip-lab/sit742](https://github.com/tulip-lab/sit742/issues)


Prepared by **SIT742 Teaching Team**

---


## Session 3X: Advanced Data wrangling

In this session, we will learn how to use Python Packages to do the data wrangling tasks on the data and files.



### Content


1. `Pandas` Basics

2. Loading `CSV` Data

3. Data Extraction through `Pandas`

4. Data Wrangling using python and also other libraries.



**Note**: The data available on those service might be changing, so you need to adjust the code to accommodate those changes.

---

In [ ]:
# Load the pandas library
import pandas as pd

## Step 1 inspect raw data

Looking at the raw data, is there anything you could find which is not quite right?

In [ ]:
# load data
df = pd.read_csv('AirCrashes.csv') # df is a dataframe, confirm with: type(df)
df.shape

In [ ]:
df.head(20)

Some points that we could see:


1.   The data's header is not correct
2.   The actual content in each column does not make sense -- different information mixed into one column.
3.   Every 15 rows, similar pattern happens
4.   There are some NaNs in the data







What if the data is read as txt?

In [ ]:
with open('AirCrashes.csv','r') as infile:
    text = infile.read()
# Print the first 680 characters from txt file
print(text[0:680])

Now can you see the pattern in the raw data?
Can you summarize the pattern?

1.png

sumamrized pattern:


1.   Directly read raw data in csv is not appropriate
1.   The data is gathered by multiple chunks of information
2.   Each chunk will have its first line as description of the aircrash
2.   There is a empty line between two different chunks of information
1.   There are total 15 features to detail each aircrash.
2.   Each feature has consistent format cross all the chunks.
3. The data seems to be comma delimited, however with missing commas after 'Incident' in the leading row for observations.

By noting those patterns, do you have a idea to do data wrangling for
transforming the current aircrash.csv to aircrash_fixed.csv with a better readibility and format?

For example:

1.   each feature could be a seperate column;
2.   each row is an individual aircrash case;
1.   Description of the aircrash will be a seperate column;
2.   Given correct format on each of the column (date / numerical / str)














## Solution for wrangling the data

We will make changes to the raw data file and save it to a file (readable via pandas) by read each line and:

1. add comma after "Incident",
2. remove blank lines, and
2. save the file to 'AirCrashes_fixed.csv'

In [ ]:
with open('AirCrashes.csv','r') as infile:
    with open('AirCrashes_fixed.csv','a') as outfile: # somewhere to write output to.
        text = infile.read()
        text = text.split('\n') # create a list to iterate over
        for line in text:
            if line == '': continue # deal with the blank lines
            if line.startswith('Incident'):
                line = line.replace('Incident','Incident,') # add the comma after 'Incident' column header
            print(line + '\n\r',file=outfile) #create a new csv with newline and carrige return characters

Now we read the new crated aircrash_fix.csv filer to pandas.

In [ ]:
data = pd.read_csv('AirCrashes_fixed.csv')
data.head(20)

Ok, it is still not quite right, but it is not far to achieve.
What to do for next?


1.   The feature and incidient are repeated every 16 rows, we need to divide it
2.   If we transpose the current data, then each feature will be a column
1.   We need to append each of the transposed line vertically as a new dataset
2.   Format the column with correct data format.





In [ ]:
# read data in to chunks and chunky_data is not a dataframe now
# the chunky_data is a text file reader object
chunky_data = pd.read_csv('AirCrashes_fixed.csv',header = None,iterator=True,chunksize=16)

In [ ]:
temp_df = []
for chunk in chunky_data:
    # transponse chunk and save the second row (the row containing values not header) to "row"
    row = chunk.T.iloc[1].values
    #print(row.values)
    # append a "row" to the temp list
    temp_df.append(row)
# given the list of list to dataframe with 16 columns
# why 16 columns? 15 features + incident description
master = pd.DataFrame(temp_df)

In [ ]:
master.head(5)

Yeahh! not far, the current master dataframe does not have head, let's add the header!

In [ ]:
column_headers = data.iloc[:,0].unique().tolist() # Get the data again
column_headers

Be careful, the incident in master dataframe is in the first column, not last.

In [ ]:
# why we could do the list + list?
new_header = ['Incident']+column_headers[:-1]
new_header

In [ ]:
master.columns = new_header # We supply it with a list of the same length
master.head() #done!

Now let's look at the data type

In [ ]:
master.dtypes

ok, we need to convert some of them to be numerical or date?

For example:

1.   Crew -- Int64
2.   Passengers -- Int64
1.   Ground -- Int64
1.   Total Dead -- Int64
2.   Latitude -- float64
2.   Longitude -- float64

Date should be in "date format" as well





In [ ]:
# Let's first create numerical format dataframe
master_num = master[['Total Dead','Crew','Passengers','Ground','Latitude','Longitude']].apply(pd.to_numeric)
master_num.dtypes

In [ ]:
# In case some of the '-' under date column has space, let's first extract the yyyymmdd out of the date string
import re
master['Date_numerical'] = master['Date'].apply(lambda x:int(re.sub('[^0-9]', '', x)))

In [ ]:
master['Date_numerical'].shape

In [ ]:
# let's find out the common pattern on Date_numerical, usually yyyy is fine, but we need to know whether it
# is mmdd or ddmm or it is mixed
for x in range(master['Date_numerical'].shape[0]):
  if int(str(master['Date_numerical'].iloc[x])[4:6]) > 12:
    print(str(master['Date_numerical'].iloc[x])+' is yyyyddmm'+' at index'+' '+str(x))
  else:
    pass



In [ ]:
# Ok, one of the row has yyyyddmm, let's change it back
# what about multiple rows has yyyyddmm? change it in a batch

master['Date_numerical'].iloc[47] = 19890919

In [ ]:
# convert yyyymmdd to yyyy-mm-dd
master['Date_conv'] = master['Date_numerical'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d',errors='coerce'))

In [ ]:
# is there any failure of the date converting?
master['Date_conv']

In [ ]:
# All done! let's reform the dataframe and save it to a readable csv!
# But we need to find out the str columns which we did not change
column_str = [x for x in new_header if x not in master_num.columns and x != 'Date']

In [ ]:
# Now let's reform the dataframe
master_final = pd.concat([master[column_str],master_num,master[['Date_conv']]],axis=1)
master_final.head(5)

We achieved our data wrangling goal on this dataset, but could you find any other issues?